#### 1. 기본 설정


- 라이브러리

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

- 불러오기 (전처리 전)

In [3]:
file_path = '../lendingClub 대출데이터셋/accepted_2007_to_2018Q4.csv'

# 필요한 컬럼만 (열= 13개)
selected_cols = [
    'loan_status',      # 타겟: 상환 여부
    'annual_inc',        # 연간 소득
    'dti',               # 총부채상환비율 (빚/소득)
    'home_ownership',    # 주거 형태 (자가, 월세 등)
    'fico_range_low',    # 신용점수 하한선
    'revol_util',        # 리볼빙 이용률
    'delinq_2yrs',       # 지난 2년간 30일 이상 연체한 횟수
    'pub_rec',           # 파산 등 공적 기록 횟수
    'loan_amnt',         # 대출 원금
    'int_rate',          # 이자율
    'term',              # 대출 기간
    'purpose',           # 대출 목적
    'grade'             # 신용 등급
]

df = pd.read_csv(file_path, usecols=selected_cols, low_memory=False)

# 행렬 개수 확인
record, columns = df.shape
print(f'행: {record}, 열: {columns}') 

행: 2260701, 열: 13


- 한글 폰트 깨짐 방지

In [ ]:
import platform
if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic') 
else:
    # 리눅스
    plt.rc('font', family='NanumGothic')

# 마이너스 기호방지
plt.rcParams['axes.unicode_minus'] = False

In [4]:
df['annual_inc'].max()

np.float64(110000000.0)

####  2. 데이터 전처리

- 결측치 확인

In [5]:
df.isnull().sum()

loan_amnt           33
term                33
int_rate            33
grade               33
home_ownership      33
annual_inc          37
loan_status         33
purpose             33
dti               1744
delinq_2yrs         62
fico_range_low      33
pub_rec             62
revol_util        1835
dtype: int64

- 결측치 제거 목록

1. loan_amnt, term **(제거)**
거의 모든 필수 컬럼에 공통적으로 33개의 결측치가 있는데 이는 시스템 오류나 빈줄이라고 판단하여 제거

2. delinq_2yrs, pub_rec, inq_last_6mths **(0으로 변환)**
대략 60여개의 결측치가 있음
신용 조회나 연체기록이 NaN인 경우 시스템상 해당 사항이 없다(0건) 일 확률이 높기때문에 0으로 대체

3. dti, revol_util, annual_inc **(제거)**
각각 1744, 1835개의 결측치가 있음 (annual은 33개)
해당 컬럼들은 리스크 판단의 핵심 근거로 다른 값으로 채우기엔 위험하며 비율 역시도 굉장히 적기때문에 제거

In [6]:
# 1. 
df.dropna(subset=['loan_amnt'], inplace=True)

# 2. 
fill_zero = ['delinq_2yrs', 'pub_rec']
df[fill_zero] = df[fill_zero].fillna(0) 

# 3. 
drop_cols = ['dti', 'revol_util', 'annual_inc']
df.dropna(subset=drop_cols, inplace=True)

# 확인
print('전처리 확인')
df.isnull().sum()

전처리 확인


loan_amnt         0
term              0
int_rate          0
grade             0
home_ownership    0
annual_inc        0
loan_status       0
purpose           0
dti               0
delinq_2yrs       0
fico_range_low    0
pub_rec           0
revol_util        0
dtype: int64

- 이상치 처리

1. 100 dti(소득대비 부채비율이 100을 넘어서면 이상치)

2. 카드 사용률(사용한도)가 100을 넘으면 100으로 바꾸기

In [7]:
df.loc[df['dti'] > 100].count()

loan_amnt         2559
term              2559
int_rate          2559
grade             2559
home_ownership    2559
annual_inc        2559
loan_status       2559
purpose           2559
dti               2559
delinq_2yrs       2559
fico_range_low    2559
pub_rec           2559
revol_util        2559
dtype: int64

In [8]:
# 1. Pub_rec: 상위 99.5% 값으로 클리핑
limit_995 = df['pub_rec'].quantile(0.995) # 분위수(0.995)
df['pub_rec'] = df['pub_rec'].clip(upper=limit_995) 
print(f"pub_rec 상한선 적용값: {limit_995}회") # 몇회 걸리지??


# 2. DTI
df = df[df['dti'] >= 0]
# 너무 큰 값은 100으로 제한
df.loc[df['dti'] > 100, 'dti'] = 100

### 취소 (고소득자를 제거하지않고 cliping하는 방법으로)
# 3. Annual_inc: 초고소득자 삭제 ------> 1000만 달러(약 130억) 이상인 행 삭제
# df = df[df['annual_inc'] <= 10000000]

# 3. annual_inc: 초고소득자 클리핑 



# 4. Revol_util: 비정상 데이터 삭제 -----> 200% 넘는 건 오류로 보고 삭제
df = df[df['revol_util'] <= 200]


# 확인
print("확인용")
print(df[['pub_rec', 'dti', 'annual_inc', 'revol_util']].describe())
print(f"남은 행: {len(df):,}건")

pub_rec 상한선 적용값: 3.0회
확인용
            pub_rec           dti    annual_inc    revol_util
count  2.257155e+06  2.257155e+06  2.257155e+06  2.257155e+06
mean   1.898066e-01  1.866527e+01  7.804233e+04  5.033807e+01
std    4.824540e-01  9.685962e+00  1.127119e+05  2.470419e+01
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
25%    0.000000e+00  1.190000e+01  4.600000e+04  3.150000e+01
50%    0.000000e+00  1.784000e+01  6.500000e+04  5.030000e+01
75%    0.000000e+00  2.449000e+01  9.300000e+04  6.940000e+01
max    3.000000e+00  1.000000e+02  1.100000e+08  1.930000e+02
남은 행: 2,257,155건


- 이상치 처리 후

In [9]:
df.loc[df['dti'] > 100].count()

loan_amnt         0
term              0
int_rate          0
grade             0
home_ownership    0
annual_inc        0
loan_status       0
purpose           0
dti               0
delinq_2yrs       0
fico_range_low    0
pub_rec           0
revol_util        0
dtype: int64

- 파생변수 

1. loan_status_binary : 이진위로 분류

- fully paid(정상 상환) = 0
- charged off(부실/연체) = 1

2. LTI : 내 연봉 대비 대출금이 얼마인가? (대출금 / 연소득)

3. 등급 수치화: A는 1등급, G는 7등급 (숫자가 클수록 위험)

In [20]:
# 1. LTI
df['LTI'] = df['loan_amnt'] / df['annual_inc']


# 2. 등급 수치화
grade_dic = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
df['grade_num'] = df['grade'].map(grade_dic)

# 확인용
df[['dti', 'revol_util', 'LTI', 'grade_num']].describe()

,dti,revol_util,LTI,grade_num
count,2.258957e+06,2.258866e+06,2.260664e+06,2.260668e+06
mean,1.882420e+01,5.033770e+01,inf,2.663888e+00
std,1.418333e+01,2.471307e+01,NaN,1.258087e+00
min,-1.000000e+00,0.000000e+00,1.612903e-04,1.000000e+00
25%,1.189000e+01,3.150000e+01,1.250000e-01,2.000000e+00
50%,1.784000e+01,5.030000e+01,2.000000e-01,3.000000e+00
75%,2.449000e+01,6.940000e+01,2.990593e-01,3.000000e+00
max,9.990000e+02,8.923000e+02,inf,7.000000e+00


In [16]:
pd.qcut(df['annual_inc'], 4, labels=['A', 'B','C', 'D'])


0          B
1          B
2          B
3          D
4          D
          ..
2260694    D
2260695    D
2260696    D
2260697    D
2260698    D
Name: annual_inc, Length: 2257155, dtype: category
Categories (4, object): ['A' < 'B' < 'C' < 'D']